In [1]:
pip install tensorflow-gpu==2.8.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install protobuf==3.20

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
import warnings
# 경고메세지 끄기
warnings.filterwarnings (action='ignore')

In [9]:
import pandas as pd
df = pd.read_csv('./total_data.csv')

In [10]:
df.head()

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61220 entries, 0 to 61219
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  61218 non-null  object
 1   name    61220 non-null  object
 2   theme   61220 non-null  object
 3   동영상 제목  61220 non-null  object
 4   댓글      61220 non-null  object
 5   댓글 작성일  61220 non-null  object
 6   PRO     61220 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


## 긍정,부정 분류 모델 적용

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import time
import re
import os
import urllib.request
from tqdm import tqdm

In [13]:
# GPU 사용 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"사용하고 있는 건? → {device}")

사용하고 있는 건? → cuda


In [14]:
# 모델 불러오기
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", from_pt=True)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

# 가중치 저장
model.save_weights("./best_model_self(2).h5")

# 가중치 로드
model.load_weights("./best_model_self(2).h5")

2023-11-27 05:42:08.653758: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-27 05:42:08.657414: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-27 05:42:08.659488: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-27 05:42:08.662136: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [15]:
# 댓글 예측 함수 정의
def predict_comments(df):
    inputs = tokenizer(df["PRO"].tolist(), return_tensors='tf', truncation=True, max_length=256, padding=True)
    inputs_dict = {key: inputs[key].numpy() for key in inputs}
    outputs = model.predict(inputs_dict)
    predicted_labels = tf.argmax(tf.nn.softmax(outputs["logits"], axis=1), axis=1)
    return predicted_labels.numpy()

In [16]:
# 진행상황 출력
def print_progress(count, total, start_time):
    percent = count / total * 100
    elapsed_time = time.time() - start_time
    print(f"{count} / {total}({percent:.2f}%) ({elapsed_time:.2f} seconds)")

# 댓글을 500개씩 split
df_split = np.array_split(df, len(df) // 500 + 1)

# 진행시간 측정
start_time = time.time()

# 각 split된 데이터프레임의 댓글 예측
predictions_list = []
for i, df_split_item in enumerate(df_split):
    predictions_list.append(predict_comments(df_split_item))
    print_progress(i + 1, len(df_split), start_time)

# 예측 결과 합치기
predictions_df = np.concatenate(predictions_list)

# 예측 결과 출력
print("댓글 예측 결과:", predictions_df)

1 / 123(0.81%) (7.18 seconds)
2 / 123(1.63%) (11.24 seconds)
3 / 123(2.44%) (13.07 seconds)
4 / 123(3.25%) (17.42 seconds)
5 / 123(4.07%) (19.35 seconds)
6 / 123(4.88%) (21.10 seconds)
7 / 123(5.69%) (22.91 seconds)
8 / 123(6.50%) (24.85 seconds)
9 / 123(7.32%) (26.62 seconds)
10 / 123(8.13%) (28.56 seconds)
11 / 123(8.94%) (30.37 seconds)
12 / 123(9.76%) (32.29 seconds)
13 / 123(10.57%) (34.44 seconds)
14 / 123(11.38%) (36.67 seconds)
15 / 123(12.20%) (38.51 seconds)
16 / 123(13.01%) (40.48 seconds)
17 / 123(13.82%) (42.32 seconds)
18 / 123(14.63%) (44.16 seconds)
19 / 123(15.45%) (46.11 seconds)
20 / 123(16.26%) (48.14 seconds)
21 / 123(17.07%) (50.08 seconds)
22 / 123(17.89%) (51.85 seconds)
23 / 123(18.70%) (53.70 seconds)
24 / 123(19.51%) (55.46 seconds)
25 / 123(20.33%) (57.41 seconds)
26 / 123(21.14%) (59.24 seconds)
27 / 123(21.95%) (61.09 seconds)
28 / 123(22.76%) (63.04 seconds)
29 / 123(23.58%) (64.87 seconds)
30 / 123(24.39%) (66.65 seconds)
31 / 123(25.20%) (68.61 seconds)

In [23]:
np.unique(predictions_df)

array([0, 1])

In [20]:
# 예측 결과를 "긍정" 또는 "부정"으로 변환하는 함수 정의
def convert_to_sentiment(predictions):
    sentiment_labels = ["중립","긍정","부정"]
    return [sentiment_labels[prediction] for prediction in predictions]

# 예측 결과를 "긍정" 또는 "부정"으로 변환
sentiments_df = convert_to_sentiment(predictions_df)

In [21]:
# 예측 결과를 데이터프레임에 추가
df["예측 감정_Selflabeling"] = sentiments_df
df["예측 감정_Selflabeling"]

0        중립
1        긍정
2        긍정
3        긍정
4        긍정
         ..
61215    중립
61216    중립
61217    중립
61218    긍정
61219    중립
Name: 예측 감정_Selflabeling, Length: 61220, dtype: object

In [22]:
df["예측 감정_Selflabeling"].value_counts()

긍정    30807
중립    30413
Name: 예측 감정_Selflabeling, dtype: int64

In [19]:
df.to_csv('Selflabeling 분석 예측 결과_231127.csv', index=False, encoding='utf-8')